In [ ]:
import torch
from transformers import BertForMaskedLM, BertTokenizer, BertModel

import csv
import logging
import time
import numpy as np

from sklearn import preprocessing
import gzip
import sys

from sklearn.cluster import KMeans

from scipy.spatial import distance
import random

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

from sklearn.mixture import GaussianMixture as GMM
from sklearn.mixture import BayesianGaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

import pandas as pd
import pickle
from sklearn.metrics import adjusted_rand_score

import networkx as nx

import os
import random
from scipy.cluster import hierarchy
import matplotlib.pyplot as plt

from tabulate import tabulate

import numpy as np

from scipy.spatial.distance import cdist

from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

from scipy.spatial import distance_matrix

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from adjustText import adjust_text


from modules.modules_ import *
from clustering.gmm import *
from clustering.kmeans import *
from embeddings.extract_bert import *
from embeddings.extract_count import *

In [ ]:
with open('analysis_count_1.pickle', 'rb') as handle:
    bf_count = pickle.load(handle)

with open('analysis_gmm_20.pickle', 'rb') as handle:
    bf_bert = pickle.load(handle)

# Onomacluster

In [ ]:
relations = {
    "chamaco": ["chico", "chamaco", "pibe"], 
    "vidriera": ["vidriera", "escaparate"],
    "volante" : ["volante", "timón"],
    "sindicar" : ["sindicar", "acusar"],
    "guagua" : ["guagua", "colectivo"],
    "amarrar" : ["amarrar", "atar"],
    "cartera" : ["cartera", "bolso"],
    "coche" : ["coche", "carro"],
    "pollera" : ["pollera", "falda"]
}

In [ ]:
onomafit = {}

for target_word in relations:
    print(target_word)
    pathTestSentences = "context_tables_final/context_table_" + target_word + ".csv"
    sentences = read_sentences(pathTestSentences, variety=None, exact_word=None)
    vectors = list(embeddings[target_word])
    
    target_vectors_count = len(vectors)
    
    n_clusters =bf_bert[target_word]["n_clusters_silhouette"]


    kmeans = KMeans(n_clusters=n_clusters).fit(np.array(vectors))
    orig_labels = kmeans.labels_    

    if n_clusters < 3:
        n_clusters *= 2

    kmeans = KMeans(n_clusters=n_clusters).fit(np.array(vectors))
    y = kmeans.labels_


    centroids = kmeans.cluster_centers_

    dist = pd.DataFrame(cdist(centroids, centroids, metric="cosine"))
    print(dist)

    indices = {i:i for i in range(len(dist))}

    for i, row in dist.iterrows():
        for j in range(i + 1, len(row)):
            if row[j] < 0.10:
                indices[j] = i

    for k,v in indices.items():
        indices[k] = indices[v]

    graph = nx.read_gpickle('../graphs/' + target_word)
    wug_clusters = get_clusters(graph)

    gold_labels = [get_cluster(wug_clusters, sentence["node"]) for sentence in sentences]
    labels = [indices[i] for i in y]

    orig_sil = 0
    if len(set(y)) > 1:
        orig_sil = silhouette_score(vectors, orig_labels)

    sil = 0
    if len(set(labels)) > 1:
        sil = silhouette_score(vectors, labels)

    labels = labels[0:target_vectors_count]
    score = adjusted_rand_score(sorted(gold_labels), sorted(labels))
    orig_score = adjusted_rand_score(sorted(gold_labels), sorted(orig_labels[0:target_vectors_count]))
    purity = purity_score(gold_labels, labels)
    orig_purity = purity_score(gold_labels, orig_labels[0:target_vectors_count])
    
    onomafit[target_word] = {
        "n_clusters_onoma":len(set(indices.values())), 
        "score":score, 
        "purity": purity, 
        "silhouette" : sil, 
        "orig_sil" : orig_sil, 
        "orig_score": orig_score, 
        "orig_pur" : orig_purity
    }

In [ ]:
l = []
scores = []
purities = []
for k in onoma_bert.keys():
    if round(onomafit[k]["silhouette"] - onomafit[k]["orig_sil"],2) > 0.20 or onomafit[k]["n_clusters_onoma"] == 1:
        l.append([k, onomafit[k]["n_clusters_onoma"], onomafit[k]["score"], onomafit[k]["purity"], round(onomafit[k]["silhouette"],2)])
        scores.append(onomafit[k]["score"])
        purities.append(onomafit[k]["purity"])
    else:
        l.append([k, bf_bert[k]["n_clusters_silhouette"], onomafit[k]["orig_score"], onomafit[k]["orig_pur"], round(onomafit[k]["orig_sil"],2)])
        scores.append(onomafit[k]["orig_score"])
        purities.append(onomafit[k]["orig_pur"])

l.append(["MEAN", "-", sum(scores)/len(scores), sum(purities)/len(purities),"-"])

print(tabulate(l, headers=["Lemma", "BEST N Cluster", "BEST ARI", "BEST Purity", "BEST Silhouette"]))

# Visualize Bert Embeddings

In [ ]:
target_word = "volante"
pathTestSentences = "context_tables_final/context_table_"+target_word+".csv"
sentences = read_sentences(pathTestSentences)
vectors = extract_embeddings(sentences)

kmeans = KMeans(n_clusters=onomafit[target_word]["n_clusters_onoma"]).fit(vectors)
labels = kmeans.labels_

In [ ]:
tsne_model = TSNE(n_components=2,early_exaggeration=12,verbose=0,metric='cosine',init='pca',n_iter=2500)
vectors_2d = tsne_model.fit_transform(vectors)

fig = plt.figure(figsize=(14,7))
alltexts = list()
labeled_points = []


tws = [sentence["target_word"] for sentence in sentences]
for i, txt in enumerate(tws):
    plt.scatter(vectors_2d[i,0], vectors_2d[i,1],s=0)
    currtext = plt.text(vectors_2d[i,0], vectors_2d[i,1], txt, family='sans-serif')
    alltexts.append(currtext)
    
numiters = adjust_text(alltexts, autoalign=False, lim=50)
plt.show()

palette = sns.color_palette("bright", len(set(labels)))
fig = plt.figure(figsize=(14,7))
sns.scatterplot(vectors_2d[:,0], vectors_2d[:,1], hue=labels, legend='full', palette=palette)
plt.show()

# Compare Best Models (BERT and Count)

In [ ]:
min_num_clusters = 1
max_num_clusters = 8

best_fit = {}

vectors_path = "model_serialization/vectors/"

with open(vectors_path + 'all_w2i.pickle', 'rb') as handle:
    w2i = pickle.load(handle)

with open(vectors_path + 'all_freq_list.pickle', 'rb') as handle:
    freq_list = pickle.load(handle)


for target_word, variety in varieties.items():
    try:
        graph = nx.read_gpickle('../graphs/' + target_word)
        wug_clusters = get_clusters(graph)
    except:
        continue  
    
    pathTestSentences = "context_tables_final/context_table_" + target_word + ".csv"

    vectors, sentences = compute_vectors(target_word, pathTestSentences, variety, vectors_path, freq_list, w2i, joint=True, exact_target_word=False)
    
    vectors = normalize(vectors, norm='l2', axis=1)


    nodes = pd.read_csv(pathTestSentences, sep="\t")["identifiers"].tolist()
    gold_labels = [get_cluster(wug_clusters, node) for node in nodes]

    n_clusters = min_num_clusters
    
    score = -1.0
    silhouette = -1.0
    best_silhouette = -1.0
    best_cluster_acc = -1.0

    best_n = 1
    best_n_sil = 1
    best_n_cluster_acc = 1

    while n_clusters < max_num_clusters:
        try:
            km = KMeans(n_clusters=n_clusters)
            km.fit(vectors)
            labels = km.labels_
            
            current_score = adjusted_rand_score(sorted(gold_labels), sorted(labels))
            if n_clusters > 1:
                silhouette = silhouette_score(vectors, labels)
            cluster_acc = cluster_accuracy(gold_labels, labels)

            if current_score > score:
                score = current_score
                best_n = n_clusters

            if silhouette > best_silhouette:
                best_silhouette = silhouette
                best_n_sil = n_clusters
            
            if cluster_acc > best_cluster_acc:
                best_cluster_acc = cluster_acc
                best_n_cluster_acc = n_clusters

            n_clusters += 1
        except Exception as e:
            print(e)
            continue

        best_fit[target_word] = {"n_clusters" : best_n, "score" : score, "n_clusters_silhouette" : best_n_sil, "silhouette_score" : best_silhouette, "cluster_accuracy": best_cluster_acc, "n_cluster_acc": best_n_cluster_acc}

with open('analysis_count_1.pickle', 'wb') as handle:
    pickle.dump(best_fit, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
min_num_clusters = 1
max_num_clusters = 8

best_fit = {}

for target_word, variety in varieties.items():
    try:
        graph = nx.read_gpickle('../graphs/' + target_word)
        wug_clusters = get_clusters(graph)
    except:
        continue  
    
    pathTestSentences = "context_tables_final/context_table_" + target_word + ".csv"

    sentences = read_sentences(pathTestSentences)

    vectors = extract_embeddings(sentences)

    nodes = pd.read_csv(pathTestSentences, sep="\t")["identifiers"].tolist()
    gold_labels = [get_cluster(wug_clusters, node) for node in nodes]

    n_clusters = min_num_clusters
    
    score = -1.0
    silhouette = -1.0
    best_silhouette = -1.0
    best_cluster_acc = -1.0

    best_n = 1
    best_n_sil = 1
    best_n_cluster_acc = 1

    while n_clusters < max_num_clusters:
        try:
            model = get_best_gmm(vectors)
            
            current_score = adjusted_rand_score(sorted(gold_labels), sorted(labels))
            if n_clusters > 1:
                silhouette = silhouette_score(vectors, labels)
                cluster_acc = purity_score(gold_labels, labels)

            if current_score > score:
                score = current_score
                best_n = n_clusters

            if silhouette > best_silhouette:
                best_silhouette = silhouette
                best_n_sil = n_clusters
            
            if cluster_acc > best_cluster_acc:
                best_cluster_acc = cluster_acc
                best_n_cluster_acc = n_clusters

            n_clusters += 1
        except Exception as e:
            print(e)
            continue
        
        best_fit[target_word] = {"n_clusters" : best_n, "score" : score, "n_clusters_silhouette" : best_n_sil, "silhouette_score" : best_silhouette, "cluster_accuracy": best_cluster_acc, "n_cluster_acc": best_n_cluster_acc}

with open('analysis_gmm_20.pickle', 'wb') as handle:
    pickle.dump(best_fit, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Visualize Statistics

In [ ]:
def plot_statistics(best_fit):
    fig, ax = plt.subplots(figsize=(14,7), squeeze=True)
    words = [k for k in best_fit.keys()]

    aris = [best_fit[w]["score"] for w in words]
    silhouettes = [best_fit[w]["silhouette_score"] for w in words]
    accuracies = [best_fit[w]["cluster_accuracy"] for w in words]

    plt.plot(words, aris, label="ARI")
    plt.plot(words, silhouettes, label="Silhouette")
    plt.plot(words, accuracies, label="Purity")
    plt.xlabel('Target Words')
    plt.legend()
    plt.xticks(rotation=65)
    plt.ylabel('Statistics')
    plt.grid()
    plt.show()

def compare_stats(best_fit, best_fit_other, field, desc1, desc2, field_desc, path=None):
    fig, ax = plt.subplots(figsize=(14,7), squeeze=True)
    words = [k for k in best_fit.keys()]
    words = words[0:-1]

    y1 = [best_fit[w][field] for w in words]
    y2 = [best_fit_other[w][field] for w in words]

    compare_lists(y1, y2, words, desc1, desc2,field_desc, path)

def compare_lists(y1, y2, words, desc1, desc2, field_desc, path=None):
    fig, ax = plt.subplots(figsize=(14,7), squeeze=True)
    
    plt.plot(words, y1, label=desc1)
    plt.plot(words, y2, label=desc2)

    plt.xlabel('Target Words')
    plt.legend()
    plt.xticks(rotation=65)
    plt.ylabel(field_desc)
    plt.grid()
    if path == None:
        plt.show()
    else:
        plt.savefig(path)

In [ ]:
plot_statistics(bf_count)
compare_stats(bf_bert, bf_count, "silhouette_score", "BERT Embeddings", "Count Vectors", "Silhouette Score", "model_vis/plots/sil.png")
